<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Klassiske_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math

In [ ]:
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [ ]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag
from oppdeling_trening_testsett import enzymfordeling

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29049 entries, 2022-10-31 17:37:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29049 non-null  int64  
 1   EnzymeType_A2  29049 non-null  int64  
 2   EnzymeType_B   29049 non-null  int64  
 3   EnzymeType_C   29049 non-null  int64  
 4   EnzymeType_D   29049 non-null  int64  
 5   EnzymeType_E   29049 non-null  int64  
 6   RawMatPercent  29049 non-null  float64
 7   NIRfat         29049 non-null  float64
 8   NIRash         29049 non-null  float64
 9   NIRwater       29049 non-null  float64
 10  TT08           29049 non-null  float64
 11  TT20           29049 non-null  float64
 12  TT12           29049 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [ ]:
data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18542 entries, 2022-11-02 00:00:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18542 non-null  int64  
 1   EnzymeType_A2  18542 non-null  int64  
 2   EnzymeType_B   18542 non-null  int64  
 3   EnzymeType_C   18542 non-null  int64  
 4   EnzymeType_D   18542 non-null  int64  
 5   EnzymeType_E   18542 non-null  int64  
 6   RawMatPercent  18542 non-null  float64
 7   NIRfat         18542 non-null  float64
 8   NIRash         18542 non-null  float64
 9   NIRwater       18542 non-null  float64
 10  TT08           18542 non-null  float64
 11  TT20           18542 non-null  float64
 12  TT12           18542 non-null  float64
 13  Mw             124 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [ ]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10507 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10507 non-null  int64  
 1   EnzymeType_A2  10507 non-null  int64  
 2   EnzymeType_B   10507 non-null  int64  
 3   EnzymeType_C   10507 non-null  int64  
 4   EnzymeType_D   10507 non-null  int64  
 5   EnzymeType_E   10507 non-null  int64  
 6   RawMatPercent  10507 non-null  float64
 7   NIRfat         10507 non-null  float64
 8   NIRash         10507 non-null  float64
 9   NIRwater       10507 non-null  float64
 10  TT08           10507 non-null  float64
 11  TT20           10507 non-null  float64
 12  TT12           10507 non-null  float64
 13  Mw             336 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [ ]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Treningsandel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening"]

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett,\
emzymtyper_testsett, treningssett_umarkert, _, treningssett\
=trening_testsett_oppdeling_enzym(rå_data,
                                  data,
                                  test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_trening_markert = enzymfordeling(rå_data,
                                           treningssett_markert)
enzymdata_test = enzymfordeling(rå_data,
                                testsett)


print("Enzymtyper Fordeling: Trening markert")
print(enzymdata_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       128  34.78
E         60  16.30
B         56  15.22
C         56  15.22
D         50  13.59
A1        18   4.89
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        32  34.78
E         15  16.30
B         14  15.22
C         14  15.22
D         13  14.13
A1         4   4.35


### RandomForestRegressor

In [ ]:
# Modelloptimalisering
rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               gjentagelser = gjentagelser,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed = random_seed)

# Lagring av resultater
rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:14:20,340] A new study created in memory with name: no-name-302b790a-1bc1-4843-98d2-1b2161a9dbae
[I 2024-03-21 13:14:22,899] Trial 0 finished with value: 1025.3827070388936 and parameters: {'n_estimators': 183, 'max_depth': 3}. Best is trial 0 with value: 1025.3827070388936.
[I 2024-03-21 13:14:24,383] Trial 1 finished with value: 823.6728044762536 and parameters: {'n_estimators': 108, 'max_depth': 4}. Best is trial 1 with value: 823.6728044762536.


{'n_estimators': 108, 'max_depth': 4}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,767.779761,0.895372,549.635263,0.077084,661.236684,0.924122,482.305122,0.068853


### KNeighborsRegressor

In [ ]:
# Modelloptimalisering
knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt=splitt,
                                               gjentagelser=gjentagelser,
                                               n_runder=n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Lagring av resultater
knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:14:25,077] A new study created in memory with name: no-name-48a86547-7326-42cd-b446-e23139bd5b64
[I 2024-03-21 13:14:25,220] Trial 0 finished with value: 644.2587830513249 and parameters: {'n_neighbors': 5, 'p': 7}. Best is trial 0 with value: 644.2587830513249.
[I 2024-03-21 13:14:25,384] Trial 1 finished with value: 636.1188613197064 and parameters: {'n_neighbors': 4, 'p': 10}. Best is trial 1 with value: 636.1188613197064.


{'n_neighbors': 4, 'p': 10}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,556.810246,0.944971,408.799193,0.060189,452.714184,0.964433,304.032683,0.046423


### SupportVectorRegressor

In [ ]:
# Modelloptimalisering
svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               treningssett,
                                               X_test,
                                               y_test,
                                               splitt=splitt,
                                               gjentagelser=gjentagelser,
                                               n_runder=n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Lagring av resultater
svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:14:25,541] A new study created in memory with name: no-name-47ca6f7e-a313-4f26-8f10-614ca6773df4
[I 2024-03-21 13:14:25,649] Trial 0 finished with value: 695.7916577371055 and parameters: {'kernel': 'linear', 'C': 97, 'degree': 16, 'gamma': 'scale'}. Best is trial 0 with value: 695.7916577371055.
[I 2024-03-21 13:14:25,752] Trial 1 finished with value: 2089.4559056203 and parameters: {'kernel': 'poly', 'C': 75, 'degree': 8, 'gamma': 'auto'}. Best is trial 0 with value: 695.7916577371055.


{'kernel': 'linear', 'C': 97, 'degree': 16, 'gamma': 'scale'}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,607.960337,0.934397,446.722042,0.068245,627.751231,0.931612,437.395093,0.065203


## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data,
                                              test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i testsett
enzymdata_test = enzymfordeling(rå_data,
                                treningssett_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.1, 0.3, 0.5, 0.7, 0.9]

# Parametervalg for kryssvalidering
splitt = 5
gjentagelser = 10

# Antall runder til optimalisering
n_runder = 100

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         60  22.30
C         54  20.07
B         52  19.33
D         48  17.84
A2        41  15.24
A1        14   5.20


### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:30:19,738] A new study created in memory with name: no-name-ccc9aeba-24c9-4edc-a595-66fce69452d4


Enzymtyper Fordeling: Treningssett markert andel: 0.1
    Frekvens  Andel
A2        13  36.11
E          6  16.67
B          5  13.89
C          5  13.89
D          5  13.89
A1         2   5.56


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2


# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_trening_markert = enzymfordeling(rå_data,
                                           trening_markert)
enzymdata_test = enzymfordeling(rå_data,
                                test)


print("Enzymtyper Fordeling: Trening markert")
print(enzymdata_trening_markert)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)



# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
    Frekvens  Andel
A2       134  36.31
E         60  16.26
C         55  14.91
B         54  14.63
D         50  13.55
A1        16   4.34
Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        26  28.57
B         16  17.58
C         15  16.48
E         15  16.48
D         13  14.29
A1         6   6.59


### RandomForestregressor

In [ ]:
# Modelloptimalisering
rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

print(parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:14:50,871] A new study created in memory with name: no-name-03d05fbb-854c-457b-928f-f1e546c7a07f
[I 2024-03-21 13:14:53,684] Trial 0 finished with value: 682.7600749432229 and parameters: {'n_estimators': 167, 'max_depth': 18}. Best is trial 0 with value: 682.7600749432229.
[I 2024-03-21 13:14:55,975] Trial 1 finished with value: 682.1892405170145 and parameters: {'n_estimators': 120, 'max_depth': 15}. Best is trial 1 with value: 682.1892405170145.


{'n_estimators': 120, 'max_depth': 15}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,707.672759,0.902668,493.914622,0.067242,215.681486,0.992045,144.9647,0.02219


### KNeighborsRegressor

In [ ]:
# Modelloptimalisering
knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

print(parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:14:57,009] A new study created in memory with name: no-name-b23cabf4-a1f3-4f70-a2f1-033912025376
[I 2024-03-21 13:15:00,135] Trial 0 finished with value: 772.1859869370127 and parameters: {'n_neighbors': 2, 'p': 16}. Best is trial 0 with value: 772.1859869370127.
[I 2024-03-21 13:15:02,875] Trial 1 finished with value: 788.1765952921304 and parameters: {'n_neighbors': 7, 'p': 2}. Best is trial 0 with value: 772.1859869370127.


{'n_neighbors': 2, 'p': 16}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,696.147802,0.905813,499.037409,0.070699,327.840116,0.981621,222.331417,0.033658


### SupoortVectorRegressor

In [ ]:
# Modelloptimalisering
svr_resultater, parametere = trening_svr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)
# Lagring av resultater
svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

print(parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:15:02,995] A new study created in memory with name: no-name-86ad2429-3f84-4def-b31f-116fa619b662
[I 2024-03-21 13:15:05,370] Trial 0 finished with value: 3674.2045694083467 and parameters: {'kernel': 'poly', 'C': 15, 'degree': 17, 'gamma': 'scale'}. Best is trial 0 with value: 3674.2045694083467.
[I 2024-03-21 13:15:07,109] Trial 1 finished with value: 1477.777759136396 and parameters: {'kernel': 'rbf', 'C': 52, 'degree': 20, 'gamma': 'scale'}. Best is trial 1 with value: 1477.777759136396.


{'kernel': 'rbf', 'C': 52, 'degree': 20, 'gamma': 'scale'}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1555.462752,0.529773,1335.057103,0.172,1457.567905,0.6367,1114.342294,0.152567


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        26  28.57
B         16  17.58
C         15  16.48
E         15  16.48
D         13  14.29
A1         6   6.59


### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:16:51,200] A new study created in memory with name: no-name-c6382ccd-d198-426e-a5e2-92b7caf9b6e1


Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
A2       112  37.84
E         48  16.22
C         43  14.53
B         42  14.19
D         39  13.18
A1        12   4.05


[I 2024-03-21 13:16:53,643] Trial 0 finished with value: 685.089252559314 and parameters: {'n_estimators': 159, 'max_depth': 14}. Best is trial 0 with value: 685.089252559314.
[I 2024-03-21 13:16:57,747] Trial 1 finished with value: 1679.8869480327198 and parameters: {'n_estimators': 159, 'max_depth': 1}. Best is trial 0 with value: 685.089252559314.
[I 2024-03-21 13:17:00,238] A new study created in memory with name: no-name-68297d4a-4b4a-4e7b-811d-9d06af73fe7f


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
A2       125  37.31
E         54  16.12
B         48  14.33
C         48  14.33
D         46  13.73
A1        14   4.18


[I 2024-03-21 13:17:03,995] Trial 0 finished with value: 1084.0923797094226 and parameters: {'n_estimators': 169, 'max_depth': 3}. Best is trial 0 with value: 1084.0923797094226.
[I 2024-03-21 13:17:06,397] Trial 1 finished with value: 684.1183948374849 and parameters: {'n_estimators': 153, 'max_depth': 14}. Best is trial 1 with value: 684.1183948374849.
[I 2024-03-21 13:17:06,889] A new study created in memory with name: no-name-e250bab6-7850-4e0c-ac43-15e356af3a5e


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
A2       134  36.31
E         60  16.26
C         55  14.91
B         54  14.63
D         50  13.55
A1        16   4.34


[I 2024-03-21 13:17:09,175] Trial 0 finished with value: 682.6542141150491 and parameters: {'n_estimators': 122, 'max_depth': 14}. Best is trial 0 with value: 682.6542141150491.
[I 2024-03-21 13:17:12,911] Trial 1 finished with value: 685.1273100194086 and parameters: {'n_estimators': 185, 'max_depth': 9}. Best is trial 0 with value: 682.6542141150491.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_estimators': 159, 'max_depth': 14}, {'n_estimators': 153, 'max_depth': 14}, {'n_estimators': 122, 'max_depth': 14}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,719.991904,0.899250,508.084755,0.069340,213.157216,0.992230,144.091494,0.022088
1,0.9,717.034417,0.900076,505.729906,0.068977,212.344803,0.992289,144.038288,0.022082
2,1.0,721.434521,0.898846,503.231677,0.068770,214.702115,0.992117,145.353453,0.022231


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:17:14,018] A new study created in memory with name: no-name-0d43dad9-5dc1-4511-9be0-354716916387
[I 2024-03-21 13:17:15,812] Trial 0 finished with value: 866.4647844181017 and parameters: {'n_neighbors': 17, 'p': 11}. Best is trial 0 with value: 866.4647844181017.
[I 2024-03-21 13:17:17,602] Trial 1 finished with value: 788.6763661643762 and parameters: {'n_neighbors': 7, 'p': 10}. Best is trial 1 with value: 788.6763661643762.
[I 2024-03-21 13:17:17,919] A new study created in memory with name: no-name-5c7165bf-2250-4423-ad17-07e4ef6f2bcf
[I 2024-03-21 13:17:19,714] Trial 0 finished with value: 799.0345006742443 and parameters: {'n_neighbors': 6, 'p': 18}. Best is trial 0 with value: 799.0345006742443.
[I 2024-03-21 13:17:21,525] Trial 1 finished with value: 740.7169641939672 and parameters: {'n_neighbors': 3, 'p': 4}. Best is trial 1 with value: 740.7169641939672.
[I 2024-03-21 13:17:21,579] A new study created in memory with name: no-name-75c59e21-7490-42b0-8acf-764

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_neighbors': 7, 'p': 10}, {'n_neighbors': 3, 'p': 4}, {'n_neighbors': 7, 'p': 19}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,752.908800,0.889827,537.197173,0.072614,503.636285,0.956625,346.736468,0.052543
1,0.9,701.825539,0.904270,491.312827,0.069256,400.157821,0.972618,268.088312,0.040008
2,1.0,762.787282,0.886917,542.120702,0.073196,506.049040,0.956208,348.132010,0.052714


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:17:26,219] A new study created in memory with name: no-name-3154e128-c4a8-4ee8-9949-68a402e07dce
[I 2024-03-21 13:17:28,449] Trial 0 finished with value: 1456.7980183617688 and parameters: {'kernel': 'rbf', 'C': 54, 'degree': 9, 'gamma': 'auto'}. Best is trial 0 with value: 1456.7980183617688.
[I 2024-03-21 13:17:30,184] Trial 1 finished with value: 1893.5305377361838 and parameters: {'kernel': 'rbf', 'C': 20, 'degree': 19, 'gamma': 'scale'}. Best is trial 0 with value: 1456.7980183617688.
[I 2024-03-21 13:17:30,475] A new study created in memory with name: no-name-1f18e407-1bc6-4be3-a1e1-e41d68a2f1f8
[I 2024-03-21 13:17:32,220] Trial 0 finished with value: 1226.5769741085387 and parameters: {'kernel': 'rbf', 'C': 78, 'degree': 9, 'gamma': 'auto'}. Best is trial 0 with value: 1226.5769741085387.
[I 2024-03-21 13:17:33,921] Trial 1 finished with value: 771.587316022096 and parameters: {'kernel': 'linear', 'C': 29, 'degree': 18, 'gamma': 'auto'}. Best is trial 1 with val

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'kernel': 'rbf', 'C': 54, 'degree': 9, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 29, 'degree': 18, 'gamma': 'auto'}, {'kernel': 'rbf', 'C': 95, 'degree': 13, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,1519.765487,0.551108,1305.669522,0.168779,1422.144749,0.654144,1085.836590,0.149456
1,0.9,671.795551,0.912287,501.026874,0.068064,657.488664,0.926076,464.123763,0.070524
2,1.0,1164.283346,0.736545,978.576917,0.127347,1045.802117,0.812972,769.841056,0.109197


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_design,
                                              test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.06
C         14  20.59
B         13  19.12
D         12  17.65
A2        10  14.71
A1         4   5.88


### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:28:40,781] A new study created in memory with name: no-name-985f868d-f83b-4d46-b63a-c50d2acd5606


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
E         54  22.41
C         48  19.92
B         47  19.50
D         43  17.84
A2        37  15.35
A1        12   4.98


[I 2024-03-21 13:28:42,135] Trial 0 finished with value: 753.5826685370812 and parameters: {'n_estimators': 102, 'max_depth': 4}. Best is trial 0 with value: 753.5826685370812.
[I 2024-03-21 13:28:44,321] Trial 1 finished with value: 746.4153389432054 and parameters: {'n_estimators': 182, 'max_depth': 4}. Best is trial 1 with value: 746.4153389432054.
[I 2024-03-21 13:28:44,934] A new study created in memory with name: no-name-e4e74cbb-49bb-4fd4-b43e-f04a13abb889


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
E         60  22.39
C         53  19.78
B         52  19.40
D         48  17.91
A2        41  15.30
A1        14   5.22


[I 2024-03-21 13:28:47,783] Trial 0 finished with value: 566.4656463662179 and parameters: {'n_estimators': 200, 'max_depth': 16}. Best is trial 0 with value: 566.4656463662179.
[I 2024-03-21 13:28:49,908] Trial 1 finished with value: 570.0764238184641 and parameters: {'n_estimators': 173, 'max_depth': 15}. Best is trial 0 with value: 566.4656463662179.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_estimators': 182, 'max_depth': 4}, {'n_estimators': 200, 'max_depth': 16}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,725.448217,0.890284,549.362300,0.074237,592.720760,0.931927,447.209073,0.061338
1,1.0,569.316992,0.932428,393.410802,0.051224,197.395757,0.992564,127.134157,0.017163


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:28:50,439] A new study created in memory with name: no-name-fced767f-5ec8-4b08-96a1-08e0de701d61
[I 2024-03-21 13:28:50,513] Trial 0 finished with value: 873.0818525707277 and parameters: {'n_neighbors': 10, 'p': 7}. Best is trial 0 with value: 873.0818525707277.
[I 2024-03-21 13:28:50,587] Trial 1 finished with value: 667.1422373073112 and parameters: {'n_neighbors': 1, 'p': 3}. Best is trial 1 with value: 667.1422373073112.
[I 2024-03-21 13:28:50,624] A new study created in memory with name: no-name-970c96e6-dc90-4c8f-ab03-782929a95275
[I 2024-03-21 13:28:50,713] Trial 0 finished with value: 643.1120621645869 and parameters: {'n_neighbors': 3, 'p': 7}. Best is trial 0 with value: 643.1120621645869.
[I 2024-03-21 13:28:50,775] Trial 1 finished with value: 718.7897161398427 and parameters: {'n_neighbors': 9, 'p': 2}. Best is trial 0 with value: 643.1120621645869.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'n_neighbors': 1, 'p': 3}, {'n_neighbors': 3, 'p': 7}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,612.253422,0.921852,437.643504,0.055949,0.000000,1.000000,0.000000,0.000000
1,1.0,585.781361,0.928464,414.948522,0.052822,380.627731,0.972353,248.515819,0.033082


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:28:50,893] A new study created in memory with name: no-name-386f2d5f-df0f-4175-9b2c-7e40545dc738
[I 2024-03-21 13:28:50,956] Trial 0 finished with value: 3635.15260345904 and parameters: {'kernel': 'poly', 'C': 76, 'degree': 19, 'gamma': 'auto'}. Best is trial 0 with value: 3635.15260345904.
[I 2024-03-21 13:28:51,016] Trial 1 finished with value: 621.7817386683273 and parameters: {'kernel': 'linear', 'C': 85, 'degree': 1, 'gamma': 'scale'}. Best is trial 1 with value: 621.7817386683273.
[I 2024-03-21 13:28:51,046] A new study created in memory with name: no-name-2dc8367e-9c4a-437d-8c04-57463ef1999e
[I 2024-03-21 13:28:51,106] Trial 0 finished with value: 2046.532054880411 and parameters: {'kernel': 'poly', 'C': 96, 'degree': 12, 'gamma': 'auto'}. Best is trial 0 with value: 2046.532054880411.
[I 2024-03-21 13:28:51,168] Trial 1 finished with value: 2231.986708613463 and parameters: {'kernel': 'rbf', 'C': 8, 'degree': 6, 'gamma': 'scale'}. Best is trial 0 with value: 2

{'trenings_andel': [0.9, 1.0], 'parametere': [{'kernel': 'linear', 'C': 85, 'degree': 1, 'gamma': 'scale'}, {'kernel': 'poly', 'C': 96, 'degree': 12, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,583.679682,0.928976,424.968959,0.057838,578.538880,0.935146,388.253253,0.052199
1,1.0,2071.161640,0.105700,1593.096024,0.248031,1766.107942,0.404783,1283.044433,0.201102


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

#Oversikt over fordelingen av enzymtyper i trening og testsett
enzymdata_test = enzymfordeling(rå_data,
                                test)
print("Enzymtyper Fordeling: Test")
print(enzymdata_test)


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.8, 0.9, 1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.39
B         13  19.40
C         13  19.40
D         12  17.91
A2        10  14.93
A1         4   5.97


### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  #Oversikt over fordelingen av enzymtyper i trening og testsett
  enzymdata_trening_markert_andel = enzymfordeling(rå_data,
                                                   treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymdata_trening_markert_andel)

  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:28:51,642] A new study created in memory with name: no-name-76079c33-3f94-4ac2-853c-defee6b8bec7


Enzymtyper Fordeling: Treningssett markert andel: 0.8
    Frekvens  Andel
E         48  22.22
C         43  19.91
B         42  19.44
D         38  17.59
A2        33  15.28
A1        12   5.56


[I 2024-03-21 13:28:54,298] Trial 0 finished with value: 996.796446243022 and parameters: {'n_estimators': 132, 'max_depth': 7}. Best is trial 0 with value: 996.796446243022.
[I 2024-03-21 13:28:56,333] Trial 1 finished with value: 978.9305605689322 and parameters: {'n_estimators': 186, 'max_depth': 17}. Best is trial 1 with value: 978.9305605689322.
[I 2024-03-21 13:28:56,865] A new study created in memory with name: no-name-2ea0089f-560e-43cb-a03b-1444e1db629d


Enzymtyper Fordeling: Treningssett markert andel: 0.9
    Frekvens  Andel
E         54  22.22
C         48  19.75
B         47  19.34
D         44  18.11
A2        37  15.23
A1        13   5.35


[I 2024-03-21 13:28:58,897] Trial 0 finished with value: 984.8385667868231 and parameters: {'n_estimators': 197, 'max_depth': 8}. Best is trial 0 with value: 984.8385667868231.
[I 2024-03-21 13:29:00,962] Trial 1 finished with value: 978.3995521683839 and parameters: {'n_estimators': 199, 'max_depth': 12}. Best is trial 1 with value: 978.3995521683839.
[I 2024-03-21 13:29:01,436] A new study created in memory with name: no-name-2d93360c-1f22-4ff7-86af-4621bab9fd37


Enzymtyper Fordeling: Treningssett markert andel: 1.0
    Frekvens  Andel
E         60  22.30
C         54  20.07
B         52  19.33
D         48  17.84
A2        41  15.24
A1        14   5.20


[I 2024-03-21 13:29:03,158] Trial 0 finished with value: 994.7573589080891 and parameters: {'n_estimators': 107, 'max_depth': 12}. Best is trial 0 with value: 994.7573589080891.
[I 2024-03-21 13:29:05,179] Trial 1 finished with value: 1197.7973181427406 and parameters: {'n_estimators': 130, 'max_depth': 3}. Best is trial 0 with value: 994.7573589080891.


{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_estimators': 186, 'max_depth': 17}, {'n_estimators': 199, 'max_depth': 12}, {'n_estimators': 107, 'max_depth': 12}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,740.681481,0.870592,546.390976,0.069058,203.151185,0.992314,129.471580,0.017359
1,0.9,747.337741,0.868256,547.977989,0.069286,205.755034,0.992116,133.804888,0.017927
2,1.0,743.625810,0.869561,547.924200,0.069294,209.105267,0.991857,134.032040,0.017900


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:29:05,858] A new study created in memory with name: no-name-bc87a1ec-9d74-4848-8f21-b2612c3f02d1
[I 2024-03-21 13:29:07,723] Trial 0 finished with value: 968.5893346090081 and parameters: {'n_neighbors': 6, 'p': 20}. Best is trial 0 with value: 968.5893346090081.
[I 2024-03-21 13:29:09,157] Trial 1 finished with value: 1132.7635977315833 and parameters: {'n_neighbors': 20, 'p': 7}. Best is trial 0 with value: 968.5893346090081.
[I 2024-03-21 13:29:09,265] A new study created in memory with name: no-name-db202cb0-a5cf-4402-9424-38a2ea1a6317
[I 2024-03-21 13:29:10,596] Trial 0 finished with value: 1038.1068280586878 and parameters: {'n_neighbors': 16, 'p': 18}. Best is trial 0 with value: 1038.1068280586878.
[I 2024-03-21 13:29:11,920] Trial 1 finished with value: 1045.9896022481926 and parameters: {'n_neighbors': 14, 'p': 10}. Best is trial 0 with value: 1038.1068280586878.
[I 2024-03-21 13:29:11,954] A new study created in memory with name: no-name-7c1071cf-8a64-45e0-b

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'n_neighbors': 6, 'p': 20}, {'n_neighbors': 16, 'p': 18}, {'n_neighbors': 10, 'p': 18}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,659.415393,0.897431,481.372914,0.064369,473.483996,0.958250,331.507051,0.043918
1,0.9,807.406886,0.846226,601.332918,0.076985,677.219021,0.914591,509.474133,0.070175
2,1.0,738.367953,0.871399,517.626906,0.065151,534.141641,0.946868,387.223152,0.051737


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:29:14,822] A new study created in memory with name: no-name-a915a1e1-1c39-40a1-a4db-672cecef0764
[I 2024-03-21 13:29:16,184] Trial 0 finished with value: 2586.407970864863 and parameters: {'kernel': 'poly', 'C': 97, 'degree': 17, 'gamma': 'auto'}. Best is trial 0 with value: 2586.407970864863.
[I 2024-03-21 13:29:17,484] Trial 1 finished with value: 1826.441445888693 and parameters: {'kernel': 'poly', 'C': 80, 'degree': 5, 'gamma': 'auto'}. Best is trial 1 with value: 1826.441445888693.
[I 2024-03-21 13:29:17,596] A new study created in memory with name: no-name-aa98a472-bf94-4a35-9ce7-7f694b378bd3
[I 2024-03-21 13:29:19,286] Trial 0 finished with value: 2042.7824874242644 and parameters: {'kernel': 'rbf', 'C': 13, 'degree': 16, 'gamma': 'auto'}. Best is trial 0 with value: 2042.7824874242644.
[I 2024-03-21 13:29:21,133] Trial 1 finished with value: 2020.279632066457 and parameters: {'kernel': 'poly', 'C': 41, 'degree': 7, 'gamma': 'auto'}. Best is trial 1 with value: 

{'trenings_andel': [0.8, 0.9, 1.0], 'parametere': [{'kernel': 'poly', 'C': 80, 'degree': 5, 'gamma': 'auto'}, {'kernel': 'poly', 'C': 41, 'degree': 7, 'gamma': 'auto'}, {'kernel': 'linear', 'C': 45, 'degree': 1, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.8,1575.799148,0.414267,1289.134601,0.180108,1625.018080,0.508229,1261.260767,0.196552
1,0.9,1745.068875,0.281672,1412.650884,0.203819,1897.447764,0.329519,1502.277017,0.236005
2,1.0,611.913540,0.911676,459.225608,0.057461,599.773338,0.933008,403.542125,0.053797


# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:22:21,301] A new study created in memory with name: no-name-02f3a405-c19b-4e71-8dde-a72fc91754c2
[I 2024-03-21 13:22:22,136] Trial 0 finished with value: 1702.246977590236 and parameters: {'n_estimators': 121, 'max_depth': 1}. Best is trial 0 with value: 1702.246977590236.
[I 2024-03-21 13:22:23,110] Trial 1 finished with value: 887.8145148984231 and parameters: {'n_estimators': 110, 'max_depth': 13}. Best is trial 1 with value: 887.8145148984231.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 110, 'max_depth': 13}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,928.825421,0.846876,617.149392,0.088188,322.910672,0.981905,220.395012,0.03278


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:22:23,429] A new study created in memory with name: no-name-939df8f6-85d6-46cb-b3d0-f5fc2293c694
[I 2024-03-21 13:22:23,506] Trial 0 finished with value: 1103.2080519901583 and parameters: {'n_neighbors': 1, 'p': 10}. Best is trial 0 with value: 1103.2080519901583.
[I 2024-03-21 13:22:23,587] Trial 1 finished with value: 990.5908992620218 and parameters: {'n_neighbors': 10, 'p': 6}. Best is trial 1 with value: 990.5908992620218.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 10, 'p': 6}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,908.193522,0.853603,677.188552,0.098546,816.815255,0.884215,619.653824,0.091204


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:22:23,673] A new study created in memory with name: no-name-21710ad7-55a3-4182-b434-32cb4e0217d6
[I 2024-03-21 13:22:23,734] Trial 0 finished with value: 2313.6718134840667 and parameters: {'kernel': 'poly', 'C': 91, 'degree': 7, 'gamma': 'auto'}. Best is trial 0 with value: 2313.6718134840667.
[I 2024-03-21 13:22:23,814] Trial 1 finished with value: 1462.141498016843 and parameters: {'kernel': 'rbf', 'C': 97, 'degree': 5, 'gamma': 'auto'}. Best is trial 1 with value: 1462.141498016843.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'rbf', 'C': 97, 'degree': 5, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1221.883286,0.735007,891.675945,0.131318,1162.657633,0.765411,902.073888,0.131464


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:22:24,204] A new study created in memory with name: no-name-17ccd0f7-b993-4fb3-9b85-bc78e1cb68a7
[I 2024-03-21 13:22:26,422] Trial 0 finished with value: 1024.3608378063373 and parameters: {'n_estimators': 138, 'max_depth': 20}. Best is trial 0 with value: 1024.3608378063373.
[I 2024-03-21 13:22:28,657] Trial 1 finished with value: 1241.050913114445 and parameters: {'n_estimators': 182, 'max_depth': 3}. Best is trial 0 with value: 1024.3608378063373.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 138, 'max_depth': 20}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,679.319125,0.910312,530.833926,0.074957,309.187018,0.983653,217.271267,0.032429


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:22:29,065] A new study created in memory with name: no-name-f02c97c3-7742-4920-b553-d5ff9c565d7f
[I 2024-03-21 13:22:31,582] Trial 0 finished with value: 1235.5904412027248 and parameters: {'n_neighbors': 5, 'p': 3}. Best is trial 0 with value: 1235.5904412027248.
[I 2024-03-21 13:22:33,803] Trial 1 finished with value: 1153.4465446087775 and parameters: {'n_neighbors': 6, 'p': 19}. Best is trial 1 with value: 1153.4465446087775.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 6, 'p': 19}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,672.898676,0.911999,504.248478,0.070438,766.717832,0.899474,559.044033,0.082805


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:22:33,888] A new study created in memory with name: no-name-de4e6f85-e104-4392-b1ab-55ff4f04e215
[I 2024-03-21 13:22:35,600] Trial 0 finished with value: 1135.4107582174029 and parameters: {'kernel': 'linear', 'C': 39, 'degree': 10, 'gamma': 'scale'}. Best is trial 0 with value: 1135.4107582174029.
[I 2024-03-21 13:22:37,327] Trial 1 finished with value: 1266.962568418656 and parameters: {'kernel': 'rbf', 'C': 76, 'degree': 18, 'gamma': 'auto'}. Best is trial 0 with value: 1135.4107582174029.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'linear', 'C': 39, 'degree': 10, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,758.027771,0.888324,576.893846,0.08174,1099.319567,0.79334,816.851629,0.121681


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, _, trening_umarkert,\
_, trening = trening_testsett_oppdeling_enzym(rå_data,
                                              data_design_uten_NIR,
                                              test_andel=test_andel)

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening


  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt = splitt,
                                                 gjentagelser = gjentagelser,
                                                 n_runder = n_runder,
                                                 trenings_andel=trenings_andel)

  # Lagring av resultater
  rfr_resultater_enzym.loc[len(rfr_resultater_enzym)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)
# Data med resultater fra treningsandeler
rfr_resultater_enzym

[I 2024-03-21 13:22:37,451] A new study created in memory with name: no-name-dc5bc0e7-f163-4377-8939-825652f8a489
[I 2024-03-21 13:22:38,528] Trial 0 finished with value: 873.6064258760612 and parameters: {'n_estimators': 177, 'max_depth': 7}. Best is trial 0 with value: 873.6064258760612.
[I 2024-03-21 13:22:39,833] Trial 1 finished with value: 873.4092353229885 and parameters: {'n_estimators': 200, 'max_depth': 11}. Best is trial 1 with value: 873.4092353229885.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 200, 'max_depth': 11}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,654.678974,0.910646,468.330937,0.06274,307.237748,0.981987,219.39614,0.029009


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  knr_resultater_enzym.loc[len(knr_resultater_enzym)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_enzym

[I 2024-03-21 13:22:40,287] A new study created in memory with name: no-name-1db2c7d1-1e85-4b38-b73b-c631ce184084
[I 2024-03-21 13:22:40,347] Trial 0 finished with value: 1074.0345033455799 and parameters: {'n_neighbors': 4, 'p': 6}. Best is trial 0 with value: 1074.0345033455799.
[I 2024-03-21 13:22:40,413] Trial 1 finished with value: 1049.2028406390589 and parameters: {'n_neighbors': 6, 'p': 3}. Best is trial 1 with value: 1049.2028406390589.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 6, 'p': 3}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,792.768616,0.868977,608.497,0.085717,776.469517,0.884949,569.810242,0.07716


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 treningssett,
                                                 X_test,
                                                 y_test,
                                                 splitt=splitt,
                                                 gjentagelser=gjentagelser,
                                                 n_runder=n_runder,
                                                 trenings_andel=trenings_andel)


  # Lagring av resultater
  svr_resultater_enzym.loc[len(svr_resultater_enzym)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_enzym

[I 2024-03-21 13:22:40,485] A new study created in memory with name: no-name-327733d1-cee8-433b-adba-81556c04820c
[I 2024-03-21 13:22:40,539] Trial 0 finished with value: 1126.0150581457526 and parameters: {'kernel': 'linear', 'C': 92, 'degree': 12, 'gamma': 'scale'}. Best is trial 0 with value: 1126.0150581457526.
[I 2024-03-21 13:22:40,596] Trial 1 finished with value: 1143.3740782823731 and parameters: {'kernel': 'linear', 'C': 34, 'degree': 18, 'gamma': 'scale'}. Best is trial 0 with value: 1126.0150581457526.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'linear', 'C': 92, 'degree': 12, 'gamma': 'scale'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1134.824689,0.731519,779.43597,0.115715,1108.314992,0.765595,776.086334,0.108777


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
rfr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
knr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
svr_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

rfr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

knr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

svr_beste_parametere = {'trenings_andel': [],
                        'parametere': []}


# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

### RandomForestRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening



  # Modelloptimalisering
  rfr_resultater, parametere = trening_rfr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


  # Lagring av resultater
  rfr_resultater_dag.loc[len(rfr_resultater_dag)] = rfr_resultater

  rfr_beste_parametere['trenings_andel'].append(trenings_andel)
  rfr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(rfr_beste_parametere)

# Data med resultater fra treningsandeler
rfr_resultater_dag

[I 2024-03-21 13:22:40,815] A new study created in memory with name: no-name-8c1ff8e7-f72e-4180-ae47-2d95ac0d9c73
[I 2024-03-21 13:22:42,856] Trial 0 finished with value: 1132.5958533941312 and parameters: {'n_estimators': 139, 'max_depth': 15}. Best is trial 0 with value: 1132.5958533941312.
[I 2024-03-21 13:22:45,938] Trial 1 finished with value: 1130.2531289388053 and parameters: {'n_estimators': 184, 'max_depth': 10}. Best is trial 1 with value: 1130.2531289388053.


{'trenings_andel': [1.0], 'parametere': [{'n_estimators': 184, 'max_depth': 10}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,612.203553,0.911592,476.345878,0.061371,324.988079,0.980331,230.125292,0.030649


### KNeighborRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  knr_resultater, parametere = trening_knr_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             trening,
                                             X_test,
                                             y_test,
                                             splitt = splitt,
                                             n_runder = n_runder,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)

  # Lagring av resultater
  knr_resultater_dag.loc[len(knr_resultater_dag)] = knr_resultater

  knr_beste_parametere['trenings_andel'].append(trenings_andel)
  knr_beste_parametere['parametere'].append(parametere)

# Beste parametere

print(knr_beste_parametere)

# Data med resultater fra treningsandeler
knr_resultater_dag

[I 2024-03-21 13:22:46,637] A new study created in memory with name: no-name-eeec7788-13f0-4a37-bd27-955028a21819
[I 2024-03-21 13:22:48,401] Trial 0 finished with value: 1391.7536768444538 and parameters: {'n_neighbors': 11, 'p': 20}. Best is trial 0 with value: 1391.7536768444538.
[I 2024-03-21 13:22:49,887] Trial 1 finished with value: 1357.9038210173858 and parameters: {'n_neighbors': 17, 'p': 20}. Best is trial 1 with value: 1357.9038210173858.


{'trenings_andel': [1.0], 'parametere': [{'n_neighbors': 17, 'p': 20}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,871.423637,0.820875,679.366446,0.086947,1050.327395,0.794555,826.005163,0.112909


### SupportVectorRegressor

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell

      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Modelloptimalisering
  svr_resultater, parametere = trening_svr_dag(rå_data,
                                               trening_markert,
                                               trening_umarkert,
                                               trening,
                                               X_test,
                                               y_test,
                                               splitt = splitt,
                                               n_runder = n_runder,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)

  # Lagring av resultater
  svr_resultater_dag.loc[len(svr_resultater_dag)] = svr_resultater

  svr_beste_parametere['trenings_andel'].append(trenings_andel)
  svr_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(svr_beste_parametere)

# Data med resultater fra treningsandeler
svr_resultater_dag

[I 2024-03-21 13:22:49,966] A new study created in memory with name: no-name-fb24f20c-76bf-45da-bdae-ce34abb76b9b
[I 2024-03-21 13:22:51,519] Trial 0 finished with value: 2340.087235538107 and parameters: {'kernel': 'poly', 'C': 2, 'degree': 19, 'gamma': 'scale'}. Best is trial 0 with value: 2340.087235538107.
[I 2024-03-21 13:22:52,993] Trial 1 finished with value: 2159.880904938157 and parameters: {'kernel': 'poly', 'C': 16, 'degree': 13, 'gamma': 'auto'}. Best is trial 1 with value: 2159.880904938157.


{'trenings_andel': [1.0], 'parametere': [{'kernel': 'poly', 'C': 16, 'degree': 13, 'gamma': 'auto'}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,1733.931331,0.290812,1400.048954,0.203313,2097.496682,0.180688,1697.661848,0.254601
